<a href="https://colab.research.google.com/github/JungAYoung87/machinelearning/blob/master/4%EC%9E%A5_%EA%B3%BC%EC%A0%9C_2019250047_%EC%A0%95%EC%95%84%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#과제1

데이터 준비

In [ ]:
import numpy as np
import os

from sklearn import datasets
iris = datasets.load_iris()

X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

In [ ]:
X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [ ]:
np.random.seed(2042)

데이터셋 분할

In [ ]:
test_ratio = 0.2                                         # 테스트 세트 비율 = 20%
validation_ratio = 0.2                                   # 검증 세트 비율 = 20%
total_size = len(X_with_bias)                            # 전체 데이터셋 크기

test_size = int(total_size * test_ratio)                 # 테스트 세트 크기: 전체의 20%
validation_size = int(total_size * validation_ratio)     # 검증 세트 크기: 전체의 20%
train_size = total_size - test_size - validation_size    # 훈련 세트 크기: 전체의 60%

In [ ]:
rnd_indices = np.random.permutation(total_size)

In [ ]:
X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]

X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]

X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

타깃 변환

In [ ]:
y_train[:5]

array([0, 1, 2, 1, 1])

In [ ]:
def to_one_hot(y):
    n_classes = y.max() + 1                 # 클래스 수
    m = len(y)                              # 샘플 수
    Y_one_hot = np.zeros((m, n_classes))    # (샘플 수, 클래스 수) 0-벡터 생성
    Y_one_hot[np.arange(m), y] = 1          # 샘플 별로 해당 클래스의 값만 1로 변경. (넘파이 인덱싱 활용)
    return Y_one_hot

In [ ]:
y_train[:5]

array([0, 1, 2, 1, 1])

In [ ]:
to_one_hot(y_train[:5])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [ ]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

sigmoid 함수 구현

In [ ]:
def sigmoid(X):
    return 1 / (1+np.exp(-X))

def loss_func(X, t):
    
    delta = 1e-7    # log 무한대 발산 방지
    
    z = np.dot(X,W) + b
    t = sigmoid(z)
    
    # cross-entropy 
    return  -np.sum( y*np.log(t + delta) + (1-y)*np.log((1 - t)+delta ) )

In [ ]:

def numerical_derivative(f, X):
    delta_X = 1e-4 # 0.0001
    grad = np.zeros_like(X)
    
    it = np.nditer(X, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = X[idx]
        X[idx] = float(tmp_val) + delta_X
        fX1 = f(X) # f(X+delta_X)
        
        X[idx] = tmp_val - delta_X 
        fX2 = f(X) # f(X-delta_X)
        grad[idx] = (fX1 - fX2) / (2*delta_X)
        
        X[idx] = tmp_val 
        it.iternext()   
        
    return grad

In [ ]:
def error_val(X, y):
    delta = 1e-7    # log 무한대 발산 방지
    
    z = np.dot(X,W) + b
    t = sigmoid(z)
    
    # cross-entropy 
    return  -np.sum( y*np.log(t + delta) + (1-y)*np.log((1 - t)+delta ) ) 

# 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
# 입력변수 X : numpy type
def predict(X):
    
    z = np.dot(X,W) + b
    t = sigmoid(z)
    
    if t >= 0.5:
        result = 1  # True
    else:
        result = 0  # False
    
    return t, result

경사하강법 활용 훈련

In [ ]:
n_inputs = X_train.shape[1]           # 특성 수(n) + 1, 붓꽃의 경우: 특성 2개 + 1
n_outputs = len(np.unique(y_train))   # 중복을 제거한 클래스 수(K), 붓꽃의 경우: 3개

In [ ]:
Theta = np.random.randn(n_inputs, n_outputs)

In [ ]:
#  배치 경사하강법 구현
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

for iteration in range(n_iterations):     # 5001번 반복 훈련
    X = X_train.dot(Theta)
    Y_proba = sigmoid(X)
    
    if iteration % 500 == 0:              # 500 에포크마다 손실(비용) 계산해서 출력
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot     # 그레이디언트 계산.
    gradients = 1/m * X_train.T.dot(error)
    
    Theta = Theta - eta * gradients       # 파라미터 업데이트

0 2.879452675764795
500 0.8411619643205543
1000 0.7441758613299106
1500 0.6805538946495453
2000 0.6368494302871641
2500 0.6053324972843511
3000 0.5815889410900722
3500 0.5630243067213745
4000 0.548053181656526
4500 0.5356676834673233
5000 0.5252029776033043


In [ ]:
Theta

array([[ 3.64527907, -1.43720859, -3.65909431],
       [-1.23421557,  0.22083091,  0.21368026],
       [-0.86364788, -0.08601454,  1.6308143 ]])

## 검증

In [ ]:
logits = X_valid.dot(Theta)              
Y_proba = logistic(logits)
y_predict = np.array([])
for i in range(len(Y_proba)):
  if Y_proba[i] >= 0.5:
    y_predict = np.append(y_predict, 1)
  else:
    y_predict = np.append(y_predict, 0)

accuracy_score = np.mean(y_predict == y_valid)  # 정확도 계산
accuracy_score

0.9

규제가 추가된 경사하강법 활용 훈련

In [ ]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1        # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    X = X_train.dot(Theta)
    Y_proba = sigmoid(X)
    
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))  # 편향은 규제에서 제외
        loss = xentropy_loss + alpha * l2_loss        # l2 규제가 추가된 손실
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot
    l2_loss_gradients = np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]   # l2 규제 그레이디언트
    gradients = 1/m * X_train.T.dot(error) + l2_loss_gradients
    
    Theta = Theta - eta * gradients

0 0.5672411342787836
500 0.660797114029667
1000 0.6648760638738773
1500 0.6716657187332781
2000 0.6764787280572263
2500 0.6797981643157118
3000 0.6820813210563802
3500 0.6836476574493875
4000 0.6847196389348427
4500 0.6854519243996793
5000 0.6859514895832416


## 검증 세트를 이용한 성능 확인

In [ ]:
logits = X_valid.dot(Theta)              
Y_proba = logistic(logits)
y_predict = np.array([])
for i in range(len(Y_proba)):
  if Y_proba[i] >= 0.5:
    y_predict = np.append(y_predict, 1)
  else:
    y_predict = np.append(y_predict, 0)

accuracy_score = np.mean(y_predict == y_valid)  # 정확도 계산
accuracy_score

0.8333333333333334

조기 종료 추가

In [ ]:
eta = 0.1 
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1            # 규제 하이퍼파라미터
best_loss = np.infty   # 최소 손실값 기억 변수

Theta = np.random.randn(n_inputs, n_outputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    # 훈련 및 손실 계산
    X = X_train.dot(Theta)
    Y_proba = sigmoid(X)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    # 검증 세트에 대한 손실 계산
    X = X_valid.dot(Theta)
    Y_proba = sigmoid(X)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    
    # 500 에포크마다 검증 세트에 대한 손실 출력
    if iteration % 500 == 0:
        print(iteration, loss)
        
    # 에포크마다 최소 손실값 업데이트
    if loss < best_loss:
        best_loss = loss
    else:                                      # 에포크가 줄어들지 않으면 바로 훈련 종료
        print(iteration - 1, best_loss)        # 종료되지 이전 에포크의 손실값 출력
        print(iteration, loss, "조기 종료!")
        break

0 2.479710410471694
319 0.7133497302033113
320 0.713349807368472 조기 종료!


테스트 세트 평가

In [ ]:
logits = X_test.dot(Theta)
Y_proba = logistic(logits)
y_predict = np.array([])
for i in range(len(Y_proba)):
  if Y_proba[i] >= 0.5:
    y_predict = np.append(y_predict, 1)
  else:
    y_predict = np.append(y_predict, 0)


accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.7333333333333333

## 사이킷런 로지스틱 모델과 성능 비교

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="lbfgs", random_state=42)
log_reg.fit(X_train, y_train)

In [ ]:
log_reg.score(X_test,y_test)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = log_reg.predict(X_test)
accuracy_score(y_test, y_pred)

#과제2

과제 1에서 구현된 로지스틱 회귀 알고리즘에 일대다 방식을 적용하여 붓꽃에 대한 다중 클래서 분류 알고리즘을 구현하라. 단, 사이킷런을 전혀 사용하지 않아야 한다.

로지스틱 모델 2개 사용

- setosa인지 아닌지를 판단하는 모델

- verginica인지 아닌지를 판단하는 모델

versicolor일 확률은 1-setosa일 확룔-virginica일 확률로 계산

## 데이터셋 준비

In [ ]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]
y0 = (iris["target"] == 0).astype(np.int) #setosa 판단 모델을 위한 데이터셋
y1 = (iris["target"] == 2).astype(np.int) #virginica 판단 모델을 위한 데이터셋

In [ ]:
X_with_bias = np.c_[np.ones([len(X), 1]), X] #편향추가

In [ ]:
np.random.seed(2042) #일정한 결과를 위해 랜덤시드 지정

모델 훈련은 각 클래스에 대해 각각 이루어지기 때문에 데이터셋도 개별적으로 준비한다.

In [ ]:
test_ratio = 0.2                                         # 테스트 세트 비율 = 20%
validation_ratio = 0.2                                   # 검증 세트 비율 = 20%
total_size = len(X_with_bias)                            # 전체 데이터셋 크기

test_size = int(total_size * test_ratio)                 # 테스트 세트 크기: 전체의 20%
validation_size = int(total_size * validation_ratio)     # 검증 세트 크기: 전체의 20%
train_size = total_size - test_size - validation_size    # 훈련 세트 크기: 전체의 60%

In [ ]:
rnd_indices = np.random.permutation(total_size) #데이터 섞기

In [ ]:
X_train = X_with_bias[rnd_indices[:train_size]] 
y_train = y[rnd_indices[:train_size]]
y_train0 = y0[rnd_indices[:train_size]] #setosa에 대한 라벨
y_train1 = y1[rnd_indices[:train_size]] #virginica에 대한 라벨

X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
y_valid0 = y0[rnd_indices[train_size:-test_size]] #setosa에 대한 검증세트 라벨
y_valid1 = y1[rnd_indices[train_size:-test_size]] #virginica에 대한 검증세트 라벨

X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [ ]:
n_inputs = X_train.shape[1]
Theta0 = np.random.randn(n_inputs) #setosa 판단모델에 쓰이는 세타값
Theta1 = np.random.randn(n_inputs) #virginica 판단모델에 쓰이는 세타값

**setosa 판별 로지스틱 회귀 모델**

In [ ]:
eta = 0.1 
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.5            # 규제 하이퍼파라미터
best_loss0 = np.infty   # 최소 손실값 기억 변수

Theta0 = np.random.randn(n_inputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    # 훈련 및 손실 계산
    logits0 = X_train.dot(Theta0)
    Y_proba0 = logistic(logits0)
    error = Y_proba0 - y_train0
    gradients0 = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1]), alpha * Theta0[1:]]
    Theta0 = Theta0 - eta * gradients0

    # 검증 세트에 대한 손실 계산
    logits0 = X_valid.dot(Theta0)
    Y_proba0 = logistic(logits0)
    xentropy_loss0 = -np.mean(np.sum((y_valid0*np.log(Y_proba0 + epsilon) + (1-y_valid0)*np.log(1 - Y_proba0 + epsilon))))
    l2_loss0 = 1/2 * np.sum(np.square(Theta0[1:]))
    loss0 = xentropy_loss0 + alpha * l2_loss0
    
    # 500 에포크마다 검증 세트에 대한 손실 출력
    if iteration % 500 == 0:
        print(iteration, loss0)
        
    # 에포크마다 최소 손실값 업데이트
    if loss0 < best_loss0:
        best_loss0 = loss0
    else:                                      # 에포크가 줄어들지 않으면 바로 훈련 종료
        print(iteration - 1, best_loss0)        # 종료되기 이전 에포크의 손실값 출력
        print(iteration, loss0, "조기 종료!")
        break

0 20.540019459712514
500 7.744571615343959
1000 7.672989036271927
1500 7.668592640555666
2000 7.668314272027711
2500 7.668296612120626
3000 7.668295491624586
3500 7.668295420530142
4000 7.668295416019264
4500 7.668295415733049
5000 7.668295415714894


**virginica 판별 로지스틱 회귀 모델**

In [ ]:
eta = 0.1 
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.5            # 규제 하이퍼파라미터
best_loss1 = np.infty   # 최소 손실값 기억 변수

Theta1 = np.random.randn(n_inputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    # 훈련 및 손실 계산
    logits1 = X_train.dot(Theta1)
    Y_proba1 = logistic(logits1)
    error = Y_proba1 - y_train1
    gradients1 = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1]), alpha * Theta1[1:]]
    Theta1 = Theta1 - eta * gradients1

    # 검증 세트에 대한 손실 계산
    logits1 = X_valid.dot(Theta1)
    Y_proba1 = logistic(logits1)
    xentropy_loss1 = -np.mean(np.sum((y_valid1*np.log(Y_proba1 + epsilon) + (1-y_valid1)*np.log(1 - Y_proba1 + epsilon))))
    l2_loss1 = 1/2 * np.sum(np.square(Theta1[1:]))
    loss1 = xentropy_loss1 + alpha * l2_loss1
    
    # 500 에포크마다 검증 세트에 대한 손실 출력
    if iteration % 500 == 0:
        print(iteration, loss1)
        
    # 에포크마다 최소 손실값 업데이트
    if loss1 < best_loss1:
        best_loss1 = loss1
    else:                                      # 에포크가 줄어들지 않으면 바로 훈련 종료
        print(iteration - 1, best_loss1)        # 종료되기 이전 에포크의 손실값 출력
        print(iteration, loss1, "조기 종료!")
        break

0 45.38818486389959
500 12.482904005693054
1000 11.947222069327108
1500 11.864096195806566
2000 11.849273910674974
2500 11.846566475123907
3000 11.846069764314986
3500 11.845978563684064
4000 11.845961815948371
4500 11.845958740374874
5000 11.845958175570198


## 테스트셋 적용

위에서 구한 두 개의 세타값을 이용하여

setosa일 확률, virginica일 확률, versicolor일 확률 중 가장 높은 것을 채택하여 분류를 진행한다.



In [ ]:
logits = X_test.dot(Theta0) #setosa에 대한 확률값 추정  
setosa_proba = logistic(logits)

logits = X_test.dot(Theta1) #virginica에 대한 확률값 추정 
virginica_proba = logistic(logits)

y_predict = np.array([])
for i in range(len(Y_proba0)):
  prob_list = [[setosa_proba[i], 0], [1-setosa_proba[i]-virginica_proba[i], 1], [virginica_proba[i], 2]]
  prob_list.sort(reverse=True) #가장 높은 확률이 가장 앞으로 오게끔 정렬해준다.
  y_predict = np.append(y_predict, prob_list[0][1]) #가장 확률이 높았던 것을 예측값으로 결정한다.

In [ ]:
accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.9333333333333333

## 사이킷런의 로지스틱 모델과의 성능 비교



In [ ]:
from sklearn.linear_model import LogisticRegression
multi_log_reg = LogisticRegression(solver='newton-cg', random_state=42).fit(X_train,y_train)

multi_log_reg.score(X_test,y_test)

직접 구현한 코드와 사이킷런에 내장되어있는 로지스틱 모델과 성능이 같음을 확인할 수 있다.

#과제3